# Case Study from Youtube https://www.youtube.com/watch?v=xArNPFG_nq0&list=WL&index=28&t=185s

In [60]:
import pandas as pd;
import matplotlib.pyplot as py;
import numpy as np;

In [23]:
df = pd.read_csv('./data/singapore-residents data.csv')
df[df['year'] == 1960]

,year,level_1,value
0,1960,Total Residents,1646400
1,1960,Total Male Residents,859600
2,1960,Total Female Residents,786800
3,1960,Total Malays,232200
4,1960,Total Male Malays,121000
5,1960,Total Female Malays,111200
6,1960,Total Chinese,1251600
7,1960,Total Male Chinese,636000
8,1960,Total Female Chinese,615600
9,1960,Total Indians,133200


# Normalize data by removing calculated columns

In [35]:
idx = df[df['level_1'].str.contains('Residents') | df['level_1'].str.contains('Total Malays') | df['level_1'].str.contains('Total Chinese') | 
         df['level_1'].str.contains('Total Indians') | df['level_1'].str.contains('Other Ethnic Groups \(Total\)')].index
df.drop(idx, inplace= True)
df.head(10)

,year,level_1,value
4,1960,Total Male Malays,121000
5,1960,Total Female Malays,111200
7,1960,Total Male Chinese,636000
8,1960,Total Female Chinese,615600
10,1960,Total Male Indians,88000
11,1960,Total Female Indians,45200
13,1960,Other Ethnic Groups (Males),14600
14,1960,Other Ethnic Groups (Females),14800
19,1961,Total Male Malays,125700
20,1961,Total Female Malays,116800


# Turn rows into Columns using Pivot Table

In [53]:
df = df.pivot_table(index='year', columns=['level_1'], values=['value'])
df.columns

MultiIndex([('value', 'Other Ethnic Groups (Females)'),
            ('value',   'Other Ethnic Groups (Males)'),
            ('value',          'Total Female Chinese'),
            ('value',          'Total Female Indians'),
            ('value',           'Total Female Malays'),
            ('value',            'Total Male Chinese'),
            ('value',            'Total Male Indians'),
            ('value',             'Total Male Malays')],
           names=[None, 'level_1'])

# Reorder columns

In [54]:
df.iloc[:,[7,4,6,3,5,2,1,0]].head(10)

value                                         \
level_1 Total Male Malays Total Female Malays Total Male Indians   
year                                                               
1960             121000.0            111200.0            88000.0   
1961             125700.0            116800.0            88800.0   
1962             129300.0            121000.0            89000.0   
1963             132000.0            125200.0            87900.0   
1964             136700.0            130300.0            87400.0   
1965             141400.0            135600.0            88000.0   
1966             146500.0            141200.0            87200.0   
1967             150400.0            145400.0            87500.0   
1968             152600.0            147900.0            88200.0   
1969             155600.0            150400.0            87900.0   

                                                                      \
level_1 Total Female Indians Total Male Chinese Total Female Chinese   
year                                                                   
1960                 45200.0           636000.0             615600.0   
1961                 46900.0           657600.0             637400.0   
1962                 48100.0           676500.0             656500.0   
1963                 49500.0           694200.0             675100.0   
1964                 51000.0           712300.0             693200.0   
1965                 52500.0           729500.0             710000.0   
1966                 54000.0           746700.0             727100.0   
1967                 55400.0           762100.0             742300.0   
1968                 56400.0           774700.0             756700.0   
1969                 57200.0           784900.0             770100.0   

                                                                   
level_1 Other Ethnic Groups (Males) Other Ethnic Groups (Females)  
year                                                               
1960                        14600.0                       14800.0  
1961                        14400.0                       14800.0  
1962                        14700.0                       15100.0  
1963                        15200.0                       15900.0  
1964                        15100.0                       15600.0  
1965                        14900.0                       15000.0  
1966                        15400.0                       16300.0  
1967                        17400.0                       17100.0  
1968                        18600.0                       16900.0  
1969                        18500.0                       17900.0

# Find the Growth Rate

In [65]:
maleMalaysGR = (df['value']['Total Male Malays']).pct_change(periods=1) * 100
maleMalaysGR.head(5)

year
1960         NaN
1961    3.884298
1962    2.863962
1963    2.088167
1964    3.560606
Name: Total Male Malays, dtype: float64

In [66]:
np.mean(maleMalaysGR)

1.38279784158756